In [13]:
%load_ext autoreload
%autoreload 2

In [2]:
from U_net import U_net
import read_img
import Compute_IOU

X_trian,Y_train = read_img.read_train()
X_test,test_id = read_img.read_test()

model,results = U_net.u_net_model(X_trian,Y_train)


Getting train images and masks ... 


100%|██████████| 670/670 [02:42<00:00,  4.12it/s]


initializing U Net model.......
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 256, 256, 3)   0                                            
____________________________________________________________________________________________________
lambda_1 (Lambda)                (None, 256, 256, 3)   0           input_1[0][0]                    
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 256, 256, 64)  1792        lambda_1[0][0]                   
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 256, 256, 64)  0           conv2d_1[0][0]                   
___________________________________________________________

NameError: name 'label' is not defined

KeyboardInterrupt: 

#### predict

In [ ]:
model = load_model('model-finalproject.h5', custom_objects={'mean_iou': mean_iou})
preds_train = model.predict(X_train[:int(X_train.shape[0]*0.9)], verbose=1)
preds_val = model.predict(X_train[int(X_train.shape[0]*0.9):], verbose=1)
preds_test = model.predict(X_test, verbose=1)

# Threshold predictions
preds_train_t = (preds_train > 0.5).astype(np.uint8)
preds_val_t = (preds_val > 0.5).astype(np.uint8)
preds_test_t = (preds_test > 0.5).astype(np.uint8)

# Create list of upsampled test masks
preds_test_upsampled = []
for i in range(len(preds_test)):
    preds_test_upsampled.append(resize(np.squeeze(preds_test[i]),(sizes_test[i][0], sizes_test[i][1]), 
                                       mode='constant', preserve_range=True))
    
import run_length_encoding

rles = run_length_encoding.to_runlength_enciding(preds_test_upsampled,test_id)


### create submission file
sub = pd.DataFrame()
sub['ImageId'] = new_test_ids
sub['EncodedPixels'] = pd.Series(rles).apply(lambda x: ' '.join(str(y) for y in x))
sub.to_csv('finalproject_run_01.csv', index=False)